<a href="https://colab.research.google.com/github/cmosguy/Cachet/blob/0.1/experimental/Adam/yolov1/siim_ibad_yolov1.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
import os
if 'COLAB_GPU' in os.environ:
  from google.colab import drive
  drive.mount('/content/drive')

Mounted at /content/drive


In [6]:
import os

if 'COLAB_GPU' in os.environ:
  print("I'm running on Colab")
  !pip install --upgrade --force-reinstall --no-deps kaggle
  ! mkdir -p ~/.kaggle
  ! cp /content/drive/MyDrive/kaggle/kaggle.json ~/.kaggle
  ! chmod 600 ~/.kaggle/kaggle.json
  ##-- do this download once and store it on your google drive
  # dataset='siim-covid19-detection'
  # ! cd kaggle/input/{dataset} && kaggle competitions download -c {dataset}
  # ! cp /content/drive/MyDrive/kaggle/{dataset}.zip .
  # ! unzip {dataset}.zip
  ! mkdir -p kaggle/input/cowhat-siim-fisabio-covid18-512
  ! cd kaggle/input/cowhat-siim-fisabio-covid18-512 && kaggle datasets download -d cmosguy/cowhat-siim-fisabio-covid18-512 && unzip cowhat-siim-fisabio-covid18-512.zip


Streaming output truncated to the last 5000 lines.
  inflating: images/images/35fd2e2f7754.png  
  inflating: images/images/3606c8d8ee4b.png  
  inflating: images/images/3607190f3f38.png  
  inflating: images/images/3615df9cfc90.png  
  inflating: images/images/361d47f4b4ea.png  
  inflating: images/images/361df3525851.png  
  inflating: images/images/3639033d59f1.png  
  inflating: images/images/36400b03d8b0.png  
  inflating: images/images/3646ad5794bf.png  
  inflating: images/images/364e2b97c131.png  
  inflating: images/images/365715af043f.png  
  inflating: images/images/365bfe0fe404.png  
  inflating: images/images/36625db2640a.png  
  inflating: images/images/3670de872788.png  
  inflating: images/images/367cc5278161.png  
  inflating: images/images/36815bf5f063.png  
  inflating: images/images/36935dd80ace.png  
  inflating: images/images/36a35a318b14.png  
  inflating: images/images/36a6c5526fef.png  
  inflating: images/images/36a80e8349a6.png  
  inflating: images/images/36

In [7]:
! pip install pydicom

     |████████████████████████████████| 1.9MB 5.8MB/s 


In [1]:
import torch.nn as nn
import torch

import ast
import os
import torchvision.transforms as transforms

import pandas as pd
from PIL import Image

#copied from sohom code
#from _typeshed import ReadableBuffer
import os
import glob
import cv2
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from pandas.core import base
import pydicom
from pydicom.pixel_data_handlers.util import apply_voi_lut
import torchvision.transforms as transforms
from PIL import Image
import pylibjpeg
import warnings
warnings.filterwarnings("ignore")
import time
from tqdm import tqdm
import timeit
import ast
import matplotlib.patches as patches
import swifter
from collections import Counter

import torch.optim as optim
import torchvision.transforms.functional as FT
from torch.utils.data import DataLoader

from sklearn.model_selection import StratifiedKFold, train_test_split

ModuleNotFoundError: ignored

In [ ]:

#yeah so why i am doing this is because after flattening and convertig cnn to neural network we need to know the
# dimension size.

HARDCODE_DICTIONARY={
    '200_200_images':9216,
    '512_512_images':65536
}
# ALADDIN PERSON CODE
#HARDCODE_DICTIONARY['200_200_images']=12288


architecture_config = [
    (7, 64, 2, 3),
    "M",
    (3, 192, 1, 1),
    "M",
    (1, 128, 1, 0),
    (3, 256, 1, 1),
    (1, 256, 1, 0),
    (3, 512, 1, 1),
    "M",
    [(1, 256, 1, 0), (3, 512, 1, 1), 4],
    (1, 512, 1, 0),
    (3, 1024, 1, 1),
    "M",
    [(1, 512, 1, 0), (3, 1024, 1, 1), 2],
    (3, 1024, 1, 1),
    (3, 1024, 2, 1),
    (3, 1024, 1, 1),
    (3, 1024, 1, 1),
]
class CNNBlock(nn.Module):
    def __init__(self, in_channels, out_channels, **kwargs):
        super(CNNBlock, self).__init__()
        self.conv=nn.Conv2d(in_channels, out_channels, bias=False, **kwargs)
        self.batchnorm=nn.BatchNorm2d(out_channels)
        self.leakyrelu=nn.LeakyReLU(0,1)

    def forward(self, x):
        x=self.conv(x)
        x=self.batchnorm(x)
        x=self.leakyrelu(x)
        return x


class Yolov1(nn.Module):
    def __init__(self, in_channels=3, image_shape='200_200_images', **kwargs):
        super(Yolov1, self).__init__()
        self.architecture=architecture_config
        self.in_channels=in_channels
        self.image_shape=image_shape
        #darknet is a framework 
        self.darknet = self._create_conv_layers(self.architecture)
        self.fcs=self._create_fcs(**kwargs)

    def forward(self,x):
        x=self.darknet(x)
        x=torch.flatten(x, start_dim=1)
        x=self.fcs(x)
        return x

    def _create_conv_layers(self, architecture):
        layers=[]
        in_channels=self.in_channels
        for x in architecture:
            if type(x)==tuple:
                layers+=[CNNBlock(
                    in_channels,
                    out_channels=x[1],
                    kernel_size=x[0],
                    stride=x[2],
                    padding=x[3]
                )]
                in_channels=x[1]
            elif type(x)==str:
                layers+=[nn.MaxPool2d(kernel_size=2, stride=2)]
            elif type(x)==list:
                conv1=x[0] # tuple
                conv2=x[1] # tuple
                num_repeats=x[2] # integer

                for _ in range(num_repeats):
                    layers+=[
                        CNNBlock(
                            in_channels,
                            conv1[1],
                            kernel_size=conv1[0],
                            stride=conv1[2],
                            padding=conv1[3]
                        )
                    ]
                    layers+=[
                        CNNBlock(conv1[1], conv2[1],
                        kernel_size=conv2[0],
                        stride=conv2[2],
                        padding=conv2[3]
                         )
                    ]
                    in_channels=conv2[1]
        return nn.Sequential(*layers) # unpacks the list and return ot the sequential


    def _create_fcs(self, split_size, num_boxes, num_classes):
        S,B,C=split_size, num_boxes, num_classes
        print ("Image shape is ", self.image_shape)

        return nn.Sequential(
            nn.Flatten(),
            #Value changed for 512by512 and different for 100by100.

            #nn.Linear(1024*S*S, 496), # original paper 4096
            #nn.Linear(HARDCODE_DICTIONARY["200_by_200_images"], 496),
            nn.Linear(HARDCODE_DICTIONARY[self.image_shape], 100),
            nn.Dropout(0.0),
            nn.LeakyReLU(0.1),
            nn.Linear(100, S*S*(C+B*5)),     
        ) # original paper should be 4096


'''
def test(S=7, B=2, C=2):
    model=Yolov1(split_size=S, num_boxes=B, num_classes=C, in_channels=1)


    x=torch.randn((2,1,448,448 ))
    
    print (model(x).shape)

    model=Yolov1(split_size=S, num_boxes=B, num_classes=C)
    x=torch.randn((2,3,448,448))
    print (model(x).shape)    
    


#test()

'''

class CompetitionDataProcessor:
    def __init__(self,machine='LOCAL', reduce_size=(512,512)):
        self.reduced_size=reduce_size
        self.final_path=''
        self.data_path=''
        if machine=='LOCAL':
            base_path=os.getcwd()
            self.data_path=os.path.join(base_path,'kaggledata')
            self.final_path=os.path.join(self.data_path,'train/*/*/*.dcm')
            self.final_path_test=os.path.join(self.data_path,'test/*/*/*.dcm')
        if machine=='COLAB':
            base_path='/content/gdrive/MyDrive/kaggle_data/newdata'
            self.data_path=base_path
            self.final_path=os.path.join(self.data_path,'train/*/*/*.dcm')
            self.final_path_test=os.path.join(self.data_path,'test/*/*/*.dcm')




    def get_image_list(self):
        self.image_files_train = sorted(glob.glob(self.final_path))
        self.image_files_test=sorted(glob.glob(self.final_path_test))


    def process_dataframe_train(self):
        df_image_level=pd.read_csv(os.path.join(self.data_path,'train_image_level.csv'))
        df_study_level=pd.read_csv(os.path.join(self.data_path,'train_study_level.csv'))

        # merge
        df_image_level["key"] = df_image_level["StudyInstanceUID"].apply(lambda x: x.split('_')[0])
        df_study_level["key"] = df_study_level["id"].apply(lambda x: x.split('_')[0])
        df_merged = df_image_level.merge(df_study_level, how='inner', on="key", suffixes=('_image', '_study'))

        # clean
        df_merged = df_merged[['id_image', 'id_study', 'label', 'Negative for Pneumonia', 'Typical Appearance', 'Indeterminate Appearance', 'Atypical Appearance','boxes']]
        df_merged["id_image"] = df_merged["id_image"].apply(lambda x: x.split('_')[0])
        df_merged["id_study"] = df_merged["id_study"].apply(lambda x: x.split('_')[0])

        # Add original width and height columns
        df_merged["width"] = df_merged["height"] = 0

        self.df_merged=df_merged
        self.df_merged['path']=''
        self.df_merged['path']=self.df_merged['id_image'].apply(self._id_to_path)

        print ("adding shapes now")



        self.df_merged['image_shape']=self.df_merged['path'].swifter.apply(self.get_rows_and_cols)
        return self.df_merged

    def _id_to_path(self,id):
        only_ids=[x.split('/')[-1].split('.')[0] for x in self.image_files_train]
        return self.image_files_train[only_ids.index(id)]


    def process_dataframe_test(self):
        study_ids=[]
        image_ids=[]

    def are_files_there(self, path):
        if os.path.exists(os.path.join(self.data_path, path)):
            train_path=os.path.join(self.data_path, path, 'train')
            if os.path.exists(train_path):
                all_files=os.listdir(train_path)
                return len(all_files)

        return 0



    def process_images(self):
        if self.reduced_size!='None':
            x_max=self.reduced_size[0]
            y_max=self.reduced_size[1]
            new_dir_name=str(x_max)+'_'+str(y_max)+'_images'
            if self.are_files_there(new_dir_name)>=len(self.df_merged):
                print ("already there")
            if not os.path.exists(os.path.join(self.data_path,new_dir_name)):
                os.makedirs(os.path.join(self.data_path, new_dir_name, 'train' ))
                os.makedirs(os.path.join(self.data_path, new_dir_name, 'test' ))

        

        for i,image_path in tqdm(enumerate(self.image_files_train), total=len(self.image_files_train)):
            self.image_saver(image_path,self.reduced_size[0], self.reduced_size[1], new_dir_name)

        for i,image_path in tqdm(enumerate(self.image_files_test), total=len(self.image_files_test)):
            self.image_saver(image_path, self.reduced_size[0], self.reduced_size[1], new_dir_name, type_='test')


    def image_saver(self,image_path,h,w, new_dir_name, type_='train'):
        img_id=image_path.split('/')[-1].split('.')[0]
        height=h
        width=w
        image=self.read_dicom(image_path)
        image_pillow=Image.fromarray(image)
        image_pillow_resized=transforms.Resize((height,width))(image_pillow)

        image_name=str(img_id)+'.png'
        save_path=os.path.join(self.data_path, new_dir_name,type_,image_name)
        
        image_pillow_resized.save(save_path)





    def get_rows_and_cols(self,path):
        dicom=pydicom.read_file(path)
        rows=dicom.Rows
        cols=dicom.Columns
        return (rows, cols)

    def read_dicom(self,path, voi_lut=True, fix_monochrome=True):
        dicom = pydicom.read_file(path)
        if voi_lut:
            data = apply_voi_lut(dicom.pixel_array, dicom)
        else:
            data = dicom.pixel_array
                
        if fix_monochrome and dicom.PhotometricInterpretation == "MONOCHROME1":
            data = np.amax(data) - data
            
        data = data - np.min(data)
        data = data / np.max(data)
        data = (data * 255).astype(np.uint8)
        return data


class DF_editory:
    def __init__(self, df):
        self.df=df

    def normalise_label(self):
        self.df['normalised_label_xywh']=self.df.apply(self.get_normalised_label,axis=1)
        return self.df
        

    def get_normalised_label(self,x):
        processed_label=process_label(x.label)
        bboxes=[box[2:] for box in processed_label]
        bboxes=[[float(val) for val in box] for box in bboxes]
        new_labels=[get_xywh_from_x1y1x2y2(val, x.image_shape) for val in bboxes]

        results=[]
        for i,val in enumerate(processed_label):
            label, confidence=val[0], val[1]
            new_labels[i]=[label, confidence]+new_labels[i]
            new_labels[i]=[str(v) for v in new_labels[i]]
            new_labels[i]=' '.join(new_labels[i])
        
        return (' '.join(new_labels))


class SanityCheckMyFunctions:
    def __init__(self, df, sample=10):
        self.df=df
        self.sample=sample
        self.df=self.df.sample(self.sample)
    def sanity_check(self):
        print ("Starting Sanity Check")
 
        for i,x in tqdm(self.df.iterrows(), total=len(self.df)):
            boxes=x['boxes']
            label=x['label']
            labels=process_label(label)
            labels=[val[2:] for val in labels]
            labels=[[float(val_) for val_ in val ] for val in labels]
            if type(boxes)==float:
                continue
            boxes=ast.literal_eval(boxes)

            image_path=x['path']
            image_numpy=self.read_dicom(image_path)
            image = Image.fromarray(image_numpy)
            for val in labels:

                [x,y,w,h]=get_xywh_from_x1y1x2y2(val, image_numpy.shape)
                res=get_x1y1x2y2_from_xywh([x,y,w,h], image_numpy.shape)
                val=[round(v,5) for v in val]
                if res!=val:
                    print ("There is an issue with ", x.id_image)
                    print (res,val)
                    print ("_________________________________________")
        print ("if you saw nothing, then it works.....")


    def read_dicom(self,path, voi_lut=True, fix_monochrome=True):
        dicom = pydicom.read_file(path)
        if voi_lut:
            data = apply_voi_lut(dicom.pixel_array, dicom)
        else:
            data = dicom.pixel_array
                
        if fix_monochrome and dicom.PhotometricInterpretation == "MONOCHROME1":
            data = np.amax(data) - data
            
        data = data - np.min(data)
        data = data / np.max(data)
        data = (data * 255).astype(np.uint8)
        return data


#### IMAGE_FUNCTIONS   
def get_xywh_from_x1y1x2y2(val, image_shape):
    """
    Receives coordinates in rc format x1,y1,x2,y2 as specifed in original problem
    return the x,y,w,h form NORMALISED.
    """
    #input received in x1,y1,x2,y2 format
    width=val[2]-val[0]
    height=val[3]-val[1]

    x=(val[2]+val[0])/2.0
    y=(val[3]+val[1])/2.0

    x=x/image_shape[1]
    y=y/image_shape[0]

    width=width/image_shape[1]
    height=height/image_shape[0]

    return [x,y,width,height]

def get_x1y1x2y2_from_xywh(val, image_shape):
    """
    INPUT IS THE COORDINATE x,y,w,h where x,y is the midpoint IN NORMALISED FORM
    and w,h are the width and height NORMALISED
    """
    upper_left_x = val[0] - val[2] / 2
    upper_left_y = val[1] - val[3] / 2
    
    upper_left_x*=image_shape[1]
    upper_left_y*=image_shape[0]

    lower_right_x=val[0]+val[2]/2
    lower_right_y=val[1]+val[3]/2

    lower_right_x*=image_shape[1]
    lower_right_y*=image_shape[0]
    
    res=[upper_left_x, upper_left_y, lower_right_x, lower_right_y]
    res=[round(x,5) for x in res]
    return res

def topleftxywh_fromx1y1x2y2(val):
    #input received in x1,y1,x2,y2 format
    width=val[2]-val[0]
    height=val[3]-val[1]

    x=val[0]
    y=val[1]
    return [x,y, width, height]


def process_label(x):
    splitted=x.split(' ')
    if len(splitted)==1:
        return splitted
    return [splitted[i:i+6] for i in range(0, len(splitted),6)]



def plot_image_from_xywh(image, boxes):
    """Plots predicted bounding boxes on the image"""
    im = np.array(image)
    height, width = im.shape
    fig, ax = plt.subplots(1)
    ax.imshow(im)
    print (len(boxes))
    for box in boxes:
        rect = patches.Rectangle(
            (box[0],box[1]),
            box[2] ,
            box[3] ,
            linewidth=2,
            color='y',
            fill=0,
        )
        ax.add_patch(rect)
    plt.show()

def plot_image_from_xywh_midpoint_normalised(image, boxes):
    """Plots predicted bounding boxes on the image"""
    im = np.array(image)
    height, width = im.shape
    fig, ax = plt.subplots(1)
    ax.imshow(im)
    for box in boxes:
        upper_left_x = box[0] - box[2] / 2
        upper_left_y = box[1] - box[3] / 2
        rect = patches.Rectangle(
            (upper_left_x * width, upper_left_y * height),
            box[2] * width,
            box[3] * height,
            linewidth=2,
            color="y",
            fill=0,
        )
        # Add the patch to the Axes
        ax.add_patch(rect)

    plt.show()



'''
a=CompetitionDataProcessor(machine='LOCAL',reduce_size=(200,200))
a.get_image_list()
print (dir(a))
df_train=a.process_dataframe_train()
print ("pr")

a.process_images()
'''


class YoloLoss(nn.Module):
    def __init__(self, S=7, B=2, C=2):
        super(YoloLoss, self).__init__()
        self.mse=nn.MSELoss(reduction="sum")
        self.S=S
        self.B=B
        self.C=C
        self.lambda_noobj=0.5
        self.lambda_coord=5


    def forward(self, predictions, target):
        predictions = predictions.reshape(-1, self.S, self.S, self.C + self.B * 5)

        #remember that each cell is gonna give 2 predictions so we are just finding the IOU of these 2 predictions with 
        # the target, 21:25 means that x,y,w,h or the first box
        # 26:30 means x,y,w,h for the second box
        # predictions look like this predictions[ c1,c2,....cN, p_b1, x,y,w,h, p_b2, x,y,w,h    ]
        # p_b1 and p_b2 telss us that what is teh probability of object EXISTING.

        #### CHANGE CHANGE CHANEG if you have 2 classes ONLY


        iou_b1=intersection_over_union(predictions[..., 3:7], target[..., 3:7])
        iou_b2=intersection_over_union(predictions[..., 8:12], target[..., 8:12])

        # here nothign but just unsqueezing it. in easy 
        # for more understanding of unsqueeze look at last
        ious=torch.cat([iou_b1.unsqueeze(0), iou_b2.unsqueeze(0)], dim=0)
        iou_maxes, bestbox=torch.max(ious,dim=0)  #BEST BOX woh ha jiska iou zada ha target ke SATH. 
        exists_box=target[...,2].unsqueeze(3) # identity of obj_i if there is an object in cell i

        #### for box coordinates
        ### mid point and then widht and height , we take square root of width and HEIGHT
        #box ha be ya nahi, matlab is there a class or not?
        box_predictions = exists_box *  ( 
            bestbox* predictions[...,8:12]+ (1-bestbox) * predictions[...,3:7] 
        )
        box_targets=exists_box* target[...,3:7]  # box targets 

        box_predictions[..., 2:4]= torch.sign(box_predictions[...,2:4]) *torch.sqrt(
            torch.abs( box_predictions[..., 2:4]+1e-6  )
            )
        

        # (N,S,S,4)
        box_targets[...,2:4]=torch.sqrt(box_targets[..., 2:4])


        # N,S S,4   --->  (N*S*S*4) because mean square error wants it in this shape

        box_loss=self.mse(
            torch.flatten(box_predictions, end_dim=-2), # we wanna flatten it 
            torch.flatten(box_targets, end_dim=-2)
        )




        #### for object loss

        pred_box=(
            bestbox*predictions[...,11:12]+(1-bestbox)* predictions[..., 7:8] #multoplying probability 
        )

        # (N*S*S)
        object_loss=self.mse(
            torch.flatten(exists_box*pred_box), 
            torch.flatten(exists_box*target[...,2:3])
        )



        #### for no object loss
        ### take the loss for both
        ### N,S,S,1 -----> N, S*S CONFUSED ON THIS
        no_object_loss=self.mse(
            torch.flatten( (1-exists_box)* predictions[...,2:3], start_dim=1 ),
            torch.flatten( (1-exists_box)* target[...,2:3], start_dim=1 )
        )

        no_object_loss+=self.mse(
            torch.flatten( (1-exists_box)* predictions[...,7:8], start_dim=1 ),
            torch.flatten( (1-exists_box)* target[...,2:3]     , start_dim=1 )
        )

        ### CLASS LOSS
        ### (N,S,S,20)---> (N*S*S,20)
        class_loss=self.mse(
            torch.flatten(exists_box* predictions[...,:2], end_dim=-2),
            torch.flatten(exists_box* target[...,:2], end_dim=-2)
        )

        loss=(
            self.lambda_coord * box_loss
            + object_loss
            +self.lambda_noobj* no_object_loss
            +class_loss

        )
        return loss








'''
>>> torch.cat([a,a], dim=0)
tensor([[-0.7582, -1.5476],
        [-0.7582, -1.5476]])
>>> torch.cat([a,a], dim=0).shape
torch.Size([2, 2])
>>> a.shape
torch.Size([1, 2])
>>> a
tensor([[-0.7582, -1.5476]])
>>> a.shape
torch.Size([1, 2])
>>> torch.cat([a,a], dim=0)
tensor([[-0.7582, -1.5476],
        [-0.7582, -1.5476]])
>>> torch.cat([a,a], dim=0).shape
torch.Size([2, 2])
>>> torch.cat([a.unsqueeze(0),a.unsqueeze(0)], dim=0)
tensor([[[-0.7582, -1.5476]],

        [[-0.7582, -1.5476]]])
>>> torch.cat([a.unsqueeze(0),a.unsqueeze(0)], dim=0).shape
torch.Size([2, 1, 2])

'''



class YoloLoss(nn.Module):
    def __init__(self, S=7, B=2, C=2):
        super(YoloLoss, self).__init__()
        self.mse=nn.MSELoss(reduction="sum")
        self.S=S
        self.B=B
        self.C=C
        self.lambda_noobj=0.5
        self.lambda_coord=5


    def forward(self, predictions, target):
        predictions = predictions.reshape(-1, self.S, self.S, self.C + self.B * 5)

        #remember that each cell is gonna give 2 predictions so we are just finding the IOU of these 2 predictions with 
        # the target, 21:25 means that x,y,w,h or the first box
        # 26:30 means x,y,w,h for the second box
        # predictions look like this predictions[ c1,c2,....cN, p_b1, x,y,w,h, p_b2, x,y,w,h    ]
        # p_b1 and p_b2 telss us that what is teh probability of object EXISTING.

        #### CHANGE CHANGE CHANEG if you have 2 classes ONLY


        iou_b1=intersection_over_union(predictions[..., 3:7], target[..., 3:7])
        iou_b2=intersection_over_union(predictions[..., 8:12], target[..., 8:12])

        # here nothign but just unsqueezing it. in easy 
        # for more understanding of unsqueeze look at last
        ious=torch.cat([iou_b1.unsqueeze(0), iou_b2.unsqueeze(0)], dim=0)
        iou_maxes, bestbox=torch.max(ious,dim=0)  #BEST BOX woh ha jiska iou zada ha target ke SATH. 
        exists_box=target[...,2].unsqueeze(3) # identity of obj_i if there is an object in cell i

        #### for box coordinates
        ### mid point and then widht and height , we take square root of width and HEIGHT
        #box ha be ya nahi, matlab is there a class or not?
        box_predictions = exists_box *  ( 
            bestbox* predictions[...,8:12]+ (1-bestbox) * predictions[...,3:7] 
        )
        box_targets=exists_box* target[...,3:7]  # box targets 

        box_predictions[..., 2:4]= torch.sign(box_predictions[...,2:4]) *torch.sqrt(
            torch.abs( box_predictions[..., 2:4]+1e-6  )
            )
        

        # (N,S,S,4)
        box_targets[...,2:4]=torch.sqrt(box_targets[..., 2:4])


        # N,S S,4   --->  (N*S*S*4) because mean square error wants it in this shape

        box_loss=self.mse(
            torch.flatten(box_predictions, end_dim=-2), # we wanna flatten it 
            torch.flatten(box_targets, end_dim=-2)
        )




        #### for object loss

        pred_box=(
            bestbox*predictions[...,11:12]+(1-bestbox)* predictions[..., 7:8] #multoplying probability 
        )

        # (N*S*S)
        object_loss=self.mse(
            torch.flatten(exists_box*pred_box), 
            torch.flatten(exists_box*target[...,2:3])
        )



        #### for no object loss
        ### take the loss for both
        ### N,S,S,1 -----> N, S*S CONFUSED ON THIS
        no_object_loss=self.mse(
            torch.flatten( (1-exists_box)* predictions[...,2:3], start_dim=1 ),
            torch.flatten( (1-exists_box)* target[...,2:3], start_dim=1 )
        )

        no_object_loss+=self.mse(
            torch.flatten( (1-exists_box)* predictions[...,7:8], start_dim=1 ),
            torch.flatten( (1-exists_box)* target[...,2:3]     , start_dim=1 )
        )

        ### CLASS LOSS
        ### (N,S,S,20)---> (N*S*S,20)
        class_loss=self.mse(
            torch.flatten(exists_box* predictions[...,:2], end_dim=-2),
            torch.flatten(exists_box* target[...,:2], end_dim=-2)
        )

        loss=(
            self.lambda_coord * box_loss
            + object_loss
            +self.lambda_noobj* no_object_loss
            +class_loss

        )
        return loss



def intersection_over_union(boxes_preds, boxes_labels, box_format="midpoint"):
    """
    Calculates intersection over union

    Parameters:
        boxes_preds (tensor): Predictions of Bounding Boxes (BATCH_SIZE, 4)
        boxes_labels (tensor): Correct labels of Bounding Boxes (BATCH_SIZE, 4)
        box_format (str): midpoint/corners, if boxes (x,y,w,h) or (x1,y1,x2,y2)

    Returns:
        tensor: Intersection over union for all examples
    """

    if box_format == "midpoint":
        box1_x1 = boxes_preds[..., 0:1] - boxes_preds[..., 2:3] / 2
        box1_y1 = boxes_preds[..., 1:2] - boxes_preds[..., 3:4] / 2
        box1_x2 = boxes_preds[..., 0:1] + boxes_preds[..., 2:3] / 2
        box1_y2 = boxes_preds[..., 1:2] + boxes_preds[..., 3:4] / 2
        box2_x1 = boxes_labels[..., 0:1] - boxes_labels[..., 2:3] / 2
        box2_y1 = boxes_labels[..., 1:2] - boxes_labels[..., 3:4] / 2
        box2_x2 = boxes_labels[..., 0:1] + boxes_labels[..., 2:3] / 2
        box2_y2 = boxes_labels[..., 1:2] + boxes_labels[..., 3:4] / 2

    if box_format == "corners":
        box1_x1 = boxes_preds[..., 0:1]
        box1_y1 = boxes_preds[..., 1:2]
        box1_x2 = boxes_preds[..., 2:3]
        box1_y2 = boxes_preds[..., 3:4]  # (N, 1)
        box2_x1 = boxes_labels[..., 0:1]
        box2_y1 = boxes_labels[..., 1:2]
        box2_x2 = boxes_labels[..., 2:3]
        box2_y2 = boxes_labels[..., 3:4]

    x1 = torch.max(box1_x1, box2_x1)
    y1 = torch.max(box1_y1, box2_y1)
    x2 = torch.min(box1_x2, box2_x2)
    y2 = torch.min(box1_y2, box2_y2)

    # .clamp(0) is for the case when they do not intersect
    intersection = (x2 - x1).clamp(0) * (y2 - y1).clamp(0)

    box1_area = abs((box1_x2 - box1_x1) * (box1_y2 - box1_y1))
    box2_area = abs((box2_x2 - box2_x1) * (box2_y2 - box2_y1))

    return intersection / (box1_area + box2_area - intersection + 1e-6)


def non_max_suppression(bboxes, iou_threshold, threshold, box_format="corners"):
    """
    Does Non Max Suppression given bboxes

    Parameters:
        bboxes (list): list of lists containing all bboxes with each bboxes
        specified as [class_pred, prob_score, x1, y1, x2, y2]
        iou_threshold (float): threshold where predicted bboxes is correct
        threshold (float): threshold to remove predicted bboxes (independent of IoU) 
        box_format (str): "midpoint" or "corners" used to specify bboxes

    Returns:
        list: bboxes after performing NMS given a specific IoU threshold
    """

    assert type(bboxes) == list

    bboxes = [box for box in bboxes if box[1] > threshold]
    bboxes = sorted(bboxes, key=lambda x: x[1], reverse=True)
    bboxes_after_nms = []

    while bboxes:
        chosen_box = bboxes.pop(0)

        bboxes = [
            box
            for box in bboxes
            if box[0] != chosen_box[0]
            or intersection_over_union(
                torch.tensor(chosen_box[2:]),
                torch.tensor(box[2:]),
                box_format=box_format,
            )
            < iou_threshold
        ]

        bboxes_after_nms.append(chosen_box)

    return bboxes_after_nms


def mean_average_precision(
    pred_boxes, true_boxes, iou_threshold=0.5, box_format="midpoint", num_classes=20
):
    """
    Calculates mean average precision 

    Parameters:
        pred_boxes (list): list of lists containing all bboxes with each bboxes
        specified as [train_idx, class_prediction, prob_score, x1, y1, x2, y2]
        true_boxes (list): Similar as pred_boxes except all the correct ones 
        iou_threshold (float): threshold where predicted bboxes is correct
        box_format (str): "midpoint" or "corners" used to specify bboxes
        num_classes (int): number of classes

    Returns:
        float: mAP value across all classes given a specific IoU threshold 
    """

    # list storing all AP for respective classes
    average_precisions = []

    # used for numerical stability later on
    epsilon = 1e-6

    for c in range(num_classes):
        detections = []
        ground_truths = []

        # Go through all predictions and targets,
        # and only add the ones that belong to the
        # current class c
        for detection in pred_boxes:
            if detection[1] == c:
                detections.append(detection)

        for true_box in true_boxes:
            if true_box[1] == c:
                ground_truths.append(true_box)

        # find the amount of bboxes for each training example
        # Counter here finds how many ground truth bboxes we get
        # for each training example, so let's say img 0 has 3,
        # img 1 has 5 then we will obtain a dictionary with:
        # amount_bboxes = {0:3, 1:5}
        amount_bboxes = Counter([gt[0] for gt in ground_truths])

        # We then go through each key, val in this dictionary
        # and convert to the following (w.r.t same example):
        # ammount_bboxes = {0:torch.tensor[0,0,0], 1:torch.tensor[0,0,0,0,0]}
        for key, val in amount_bboxes.items():
            amount_bboxes[key] = torch.zeros(val)

        # sort by box probabilities which is index 2
        detections.sort(key=lambda x: x[2], reverse=True)
        TP = torch.zeros((len(detections)))
        FP = torch.zeros((len(detections)))
        total_true_bboxes = len(ground_truths)
        
        # If none exists for this class then we can safely skip
        if total_true_bboxes == 0:
            continue

        for detection_idx, detection in enumerate(detections):
            # Only take out the ground_truths that have the same
            # training idx as detection
            ground_truth_img = [
                bbox for bbox in ground_truths if bbox[0] == detection[0]
            ]

            num_gts = len(ground_truth_img)
            best_iou = 0

            for idx, gt in enumerate(ground_truth_img):
                iou = intersection_over_union(
                    torch.tensor(detection[3:]),
                    torch.tensor(gt[3:]),
                    box_format=box_format,
                )

                if iou > best_iou:
                    best_iou = iou
                    best_gt_idx = idx

            if best_iou > iou_threshold:
                # only detect ground truth detection once
                if amount_bboxes[detection[0]][best_gt_idx] == 0:
                    # true positive and add this bounding box to seen
                    TP[detection_idx] = 1
                    amount_bboxes[detection[0]][best_gt_idx] = 1
                else:
                    FP[detection_idx] = 1

            # if IOU is lower then the detection is a false positive
            else:
                FP[detection_idx] = 1

        TP_cumsum = torch.cumsum(TP, dim=0)
        FP_cumsum = torch.cumsum(FP, dim=0)
        recalls = TP_cumsum / (total_true_bboxes + epsilon)
        precisions = torch.divide(TP_cumsum, (TP_cumsum + FP_cumsum + epsilon))
        precisions = torch.cat((torch.tensor([1]), precisions))
        recalls = torch.cat((torch.tensor([0]), recalls))
        # torch.trapz for numerical integration
        average_precisions.append(torch.trapz(precisions, recalls))

    return sum(average_precisions) / len(average_precisions)


def plot_image(image, boxes, i=0):
    """Plots predicted bounding boxes on the image"""
    im = np.array(image)
    height, width, _ = im.shape

    # Create figure and axes
    fig, ax = plt.subplots(1)
    # Display the image
    ax.imshow(im)

    # box[0] is x midpoint, box[2] is width
    # box[1] is y midpoint, box[3] is height

    # Create a Rectangle potch
    for box in boxes:
        box = box[2:]
        assert len(box) == 4, "Got more values than in x, y, w, h, in a box!"
        upper_left_x = box[0] - box[2] / 2
        upper_left_y = box[1] - box[3] / 2
        rect = patches.Rectangle(
            (upper_left_x * width, upper_left_y * height),
            box[2] * width,
            box[3] * height,
            linewidth=1,
            edgecolor="r",
            facecolor="none",
        )
        # Add the patch to the Axes
        ax.add_patch(rect)

    plt.savefig('sample_result_'+str(i)+'.png')

def get_bboxes(
    loader,
    model,
    iou_threshold,
    threshold,
    pred_format="cells",
    box_format="midpoint",
    device="cuda",
    S=7,
):
    all_pred_boxes = []
    all_true_boxes = []

    # make sure model is in eval before get bboxes
    model.eval()
    train_idx = 0
    #print ("In function get_bboxes for all data")


    for batch_idx, (x, labels) in tqdm(enumerate(loader), total=len(loader)):
        x = x.to(device)
        labels = labels.to(device)

        with torch.no_grad():
            predictions = model(x)

        batch_size = x.shape[0]
        true_bboxes = cellboxes_to_boxes(labels,S=S)
        bboxes = cellboxes_to_boxes(predictions, S=S)

        for idx in range(batch_size):
            nms_boxes = non_max_suppression(
                bboxes[idx],
                iou_threshold=iou_threshold,
                threshold=threshold,
                box_format=box_format,
            )


            #if batch_idx == 0 and idx == 0:
            #    plot_image(x[idx].permute(1,2,0).to("cpu"), nms_boxes)
            #    print(nms_boxes)

            for nms_box in nms_boxes:
                all_pred_boxes.append([train_idx] + nms_box)

            for box in true_bboxes[idx]:
                # many will get converted to 0 pred
                if box[1] > threshold:
                    all_true_boxes.append([train_idx] + box)

            train_idx += 1

    model.train()
    return all_pred_boxes, all_true_boxes


def convert_cellboxes(predictions, S=7):
    """
    Converts bounding boxes output from Yolo with
    an image split size of S into entire image ratios
    rather than relative to cell ratios. Tried to do this
    vectorized, but this resulted in quite difficult to read
    code... Use as a black box? Or implement a more intuitive,
    using 2 for loops iterating range(S) and convert them one
    by one, resulting in a slower but more readable implementation.
    """

    predictions = predictions.to("cpu")
    batch_size = predictions.shape[0]
    predictions = predictions.reshape(batch_size, S, S, 12)
    bboxes1 = predictions[..., 3:7]
    bboxes2 = predictions[...,8:12]
    scores = torch.cat(
        (predictions[..., 2].unsqueeze(0), predictions[..., 7].unsqueeze(0)), dim=0
    )
    best_box = scores.argmax(0).unsqueeze(-1)
    best_boxes = bboxes1 * (1 - best_box) + best_box * bboxes2

    cell_indices = torch.arange(S).repeat(batch_size, S, 1).unsqueeze(-1)
    x = 1 / S * (best_boxes[..., :1] + cell_indices)
    y = 1 / S * (best_boxes[..., 1:2] + cell_indices.permute(0, 2, 1, 3))
    w_y = 1 / S * best_boxes[..., 2:4]
    converted_bboxes = torch.cat((x, y, w_y), dim=-1)
    predicted_class = predictions[..., :2].argmax(-1).unsqueeze(-1)
    best_confidence = torch.max(predictions[..., 2], predictions[..., 7]).unsqueeze(
        -1
    )      
    converted_preds = torch.cat(
        (predicted_class, best_confidence, converted_bboxes), dim=-1
    )

    return converted_preds


def cellboxes_to_boxes(out, S=7):
    converted_pred = convert_cellboxes(out,S=S).reshape(out.shape[0], S * S, -1)
    converted_pred[..., 0] = converted_pred[..., 0].long()
    all_bboxes = []

    for ex_idx in range(out.shape[0]):
        bboxes = []

        for bbox_idx in range(S * S):
            bboxes.append([x.item() for x in converted_pred[ex_idx, bbox_idx, :]])
        all_bboxes.append(bboxes)

    return all_bboxes

def save_checkpoint(state, filename="my_checkpoint.pth.tar"):
    print("=> Saving checkpoint")
    torch.save(state, filename)


def load_checkpoint(checkpoint, model, optimizer):
    print("=> Loading checkpoint")
    model.load_state_dict(checkpoint["state_dict"])
    optimizer.load_state_dict(checkpoint["optimizer"])





'''
>>> torch.cat([a,a], dim=0)
tensor([[-0.7582, -1.5476],
        [-0.7582, -1.5476]])
>>> torch.cat([a,a], dim=0).shape
torch.Size([2, 2])
>>> a.shape
torch.Size([1, 2])
>>> a
tensor([[-0.7582, -1.5476]])
>>> a.shape
torch.Size([1, 2])
>>> torch.cat([a,a], dim=0)
tensor([[-0.7582, -1.5476],
        [-0.7582, -1.5476]])
>>> torch.cat([a,a], dim=0).shape
torch.Size([2, 2])
>>> torch.cat([a.unsqueeze(0),a.unsqueeze(0)], dim=0)
tensor([[[-0.7582, -1.5476]],

        [[-0.7582, -1.5476]]])
>>> torch.cat([a.unsqueeze(0),a.unsqueeze(0)], dim=0).shape
torch.Size([2, 1, 2])

'''


label_dict={
    'opacity':1,
    'none':0
}

class ConvertPILtoTensor(object):
    def __init__(self, transforms):
        self.transforms=transforms
    def __call__(self, img, bboxes):
        for t in self.transforms:
            img, bboxes= t(img), bboxes

        return img, bboxes

transform_to_tensor=ConvertPILtoTensor([transforms.ToTensor()])

class DataSplitterStratifiedKfold:
    def __init__(self, data_file, num_splits=4, percentage=0.2):
        self.df=pd.read_csv(data_file)
        self.num_splits=num_splits
        self.percentage=percentage
    def split_data(self):
        skf = StratifiedKFold(n_splits=self.num_splits)
        self.df['target']=self.df['label'].apply(lambda x: x.split(' ')[0])
        
        dataframes=[]
        for train_index, test_index in skf.split(self.df, self.df.target):
            dataframes.append((self.df.loc[train_index].reset_index(),self.df.loc[test_index].reset_index()))
        return dataframes
    def split_data_by_percentage(self):
        train, test = train_test_split(self.df, test_size=self.percentage)
        return train.reset_index(), test.reset_index()

class XrayDatasetTest(torch.utils.data.Dataset):
    def __init__(self, base_path , S=7,B=2, C=2, transform=None, image_shape=(512,512)):
        self.all_images_with_path=os.listdir(base_path)
        self.all_images_with_path=[path for path in self.all_images_with_path if path[-3:]=='png']
        self.transform=transform
        self.base_path=base_path
        self.B=B
        self.S=S
        self.C=C
    def __len__(self):
        return len(self.all_images_with_path)
    def __getitem__(self, index):
        image_name=self.all_images_with_path[index]
        
        img_path=os.path.join(self.base_path, image_name)
        
        image = Image.open(img_path)
        boxes='useless_for_test'
        if self.transform:
            # image = self.transform(image)
            image, boxes = self.transform(image, boxes)
        else:
            image, boxes=transform_to_tensor(image, boxes)
        # Convert To Cells
        
        return image,'None'

class XrayDataset(torch.utils.data.Dataset):
    def __init__(self, base_path ,df, S=7,B=2, C=2, transform=None, image_shape=(512,512)):
        self.df=df
        self.transform=transform
        self.base_path=base_path
        self.B=B
        self.S=S
        self.C=C
        self.df['image_shape']=self.df['image_shape'].apply(ast.literal_eval)
    def __len__(self):
        return len(self.df)
    def __getitem__(self, index):

        label=self.df.loc[index,'normalised_label_xywh']
        label_processed=process_label(label)

        classes=[box[0] for box in label_processed]
        classes=[label_dict[x] for x in classes]

        bboxes=[box[2:] for box in label_processed]

        bboxes=[[float(val) for val in box] for box in bboxes]
        boxes=[]
        for i,val in enumerate(bboxes):
            class_label=classes[i]
            x,y,w,h=val
            boxes.append([class_label,x,y,w,h])

        image_name=self.df.loc[index, 'id_image']+'.png'
        img_path=os.path.join(self.base_path, image_name)
        
        image = Image.open(img_path)
        boxes = torch.tensor(boxes)

        if self.transform:
            # image = self.transform(image)
            image, boxes = self.transform(image, boxes)
        else:
            image, boxes=transform_to_tensor(image, boxes)
        # Convert To Cells
        label_matrix = torch.zeros((self.S, self.S, self.C + 5 * self.B)) # not gonna use the last 30
        for box in boxes:
            class_label, x, y, width, height = box.tolist()
            class_label = int(class_label)

            # i,j represents the cell row and cell column
            i, j = int(self.S * y), int(self.S * x)
            x_cell, y_cell = self.S * x - j, self.S * y - i

            """
            Calculating the width and height of cell of bounding box,
            relative to the cell is done by the following, with
            width as the example:
            
            width_pixels = (width*self.image_width)
            cell_pixels = (self.image_width)
            
            Then to find the width relative to the cell is simply:
            width_pixels/cell_pixels, simplification leads to the
            formulas below.
            """
            width_cell, height_cell = (
                width * self.S,
                height * self.S,
            )

            # If no object already found for specific cell i,j
            # Note: This means we restrict to ONE object
            # per cell!
            if label_matrix[i, j, 2] == 0:
                # Set that there exists an object
                label_matrix[i, j, 2] = 1

                # Box coordinates
                box_coordinates = torch.tensor(
                    [x_cell, y_cell, width_cell, height_cell]
                )

                label_matrix[i, j, 3:7] = box_coordinates

                # Set one hot encoding for class_label
                label_matrix[i, j, class_label] = 1

        return image, label_matrix


def train_fn(train_loader, model, optimizer, loss_fn):
    loop=tqdm (train_loader, leave=True)
    mean_loss=[]
    for batch_idx, (x,y) in enumerate(train_loader):
        x,y=x.to(DEVICE), y.to(DEVICE)
        out=model(x)
        loss=loss_fn(out,y)
        mean_loss.append(loss.item())
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

        #update the progres sbar
        loop.set_postfix(loss=loss.item())

    mean_loss=sum(mean_loss)/len(mean_loss)
    print ("Mean Loss was ", mean_loss)


ModuleNotFoundError: ignored

In [ ]:
seed = 123
torch.manual_seed(seed)
#torch.autograd.set_detect_anomaly(True)

# hyperparamaeters
LEARNING_RATE=2e-5
DEVICE="cuda" if torch.cuda.is_available() else "cpu"
BATCH_SIZE=16 # cannot have as that of paper
WEIGHT_DECAY=0
# no dropout and weight decay
# we are actually overfitting, 
# we are overfitting to just check if our implementation is right
EPOCHS=100
NUM_WORKERS=2
PIN_MEMORY=True
LOAD_MODEL=False
LOAD_MODEL_FILE='ibadski_model.pth.tar'
SPLIT_SIZE=8


#GIVE YOUR DATA PATH HERE PLEASE
image_path='200_200_images'
DATA_PATH=os.path.join('/Users/ibadski/Desktop/kaggle_comp/siimfisabio','kaggledata',image_path)

#One last request :D. There is a csv file data.csv which much be present to run the model. make sure its there

CSV_FILE_PATH='/Users/ibadski/Desktop/kaggle_comp/siimfisabio/data.csv'



def main():
    print ("WE WILL BE USING "+str(DEVICE))
    model=Yolov1(split_size=SPLIT_SIZE, num_boxes=2, num_classes=2, in_channels=1, image_shape=image_path).to(DEVICE)
    optimizer=optim.Adam(
        model.parameters(), lr=LEARNING_RATE, weight_decay=WEIGHT_DECAY
    )
    loss_fn = YoloLoss(S=SPLIT_SIZE, C=2)

    if LOAD_MODEL:
        load_checkpoint(torch.load(LOAD_MODEL_FILE), model, optimizer)
    
    
    splitterobj=DataSplitterStratifiedKfold(CSV_FILE_PATH, percentage=0.2)
    dataframes_train_test=splitterobj.split_data()
    traindf,testdf=splitterobj.split_data_by_percentage()



    data_path=os.path.join(DATA_PATH,'train')
    train_dataset=XrayDataset(data_path,traindf, S=SPLIT_SIZE)
    validation_dataset=XrayDataset(data_path, testdf, S=SPLIT_SIZE)
    test_dataset=XrayDatasetTest(os.path.join(DATA_PATH,'test'), S=SPLIT_SIZE)


    train_loader=DataLoader(
        dataset=train_dataset,
        batch_size=BATCH_SIZE,
        num_workers=NUM_WORKERS,
        pin_memory=PIN_MEMORY,
        shuffle=True,
        drop_last=False # no need to update on single example
    )

    test_loader=DataLoader(
        dataset=test_dataset,
        batch_size=BATCH_SIZE,
        num_workers=NUM_WORKERS,
        pin_memory=PIN_MEMORY,
        shuffle=True,
        drop_last=False # no need to update on single example
    )

    validation_loader=DataLoader(
        dataset=validation_dataset,
        batch_size=BATCH_SIZE,
        num_workers=NUM_WORKERS,
        pin_memory=PIN_MEMORY,
        shuffle=True,
        drop_last=False # no need to update on single example
    )



    for epoch in range(EPOCHS):
        print ("Epoch: ", epoch)
        
        for x, y in train_loader:
            x = x.to(DEVICE)
            for idx in range(8):
                bboxes = cellboxes_to_boxes(model(x), S=SPLIT_SIZE)
                bboxes = non_max_suppression(bboxes[idx], iou_threshold=0.5, threshold=0.4, box_format="midpoint")
                plot_image(x[idx].permute(1,2,0).to("cpu"), bboxes, i=idx)

        

        train_fn(train_loader, model, optimizer, loss_fn)
        
        pred_boxes, target_boxes=get_bboxes(
            train_loader, model, iou_threshold=0.5, threshold=0.4, device=DEVICE,S=SPLIT_SIZE
        )
        mean_avg_prec=mean_average_precision(
            pred_boxes, target_boxes , iou_threshold=0.5, box_format="midpoint", num_classes=2
        )


        pred_boxes_val, target_boxes_val=get_bboxes(
            validation_loader, model, iou_threshold=0.5, threshold=0.4, device=DEVICE,S=SPLIT_SIZE
        )
        mean_avg_prec_val=mean_average_precision(
            pred_boxes_val, target_boxes_val , iou_threshold=0.5, box_format="midpoint", num_classes=2
        )



        print (f" Train mAP ", mean_avg_prec)
        print (f" Validation mAP ", mean_avg_prec_val)

        print ("saving model now ")
        checkpoint = {
                    "state_dict": model.state_dict(),
                    "optimizer": optimizer.state_dict(),
                    }
        save_checkpoint(checkpoint, filename=LOAD_MODEL_FILE)